In [80]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import datetime as dt

In [3]:
#df = pd.read_csv('./bank-loan-status-dataset/credit_train.csv')

In [4]:
df.columns

Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')

# Drop unnecessary columns:

In [67]:
#df = pd.read_csv('~/Downloads/LoanStats3c.csv',header = 1)
#df.to_pickle('./data/loan_data_2014.zip',compression = 'zip')

In [147]:
df = pd.read_pickle('./data/loan_data_2014.zip',compression = 'zip')

In [148]:
# add categorical variable - HARdship END FLAG EXISTS
# THIS EXISTS AS HARDSHIP_STATUS

In [149]:
df=df.drop(columns = [
        # Only looking at loans for individuals, not joint
        'annual_inc_joint',
        'application_type',
        'dti_joint',
        'verification_status_joint',
        'revol_bal_joint', 


        # Don't need specific ID info
        'id',
        'member_id',
    
        # Only care about information available pre loan
        'collection_recovery_fee',
        'collections_12_mths_ex_med',
        # Hard to classify
        'emp_title',
    
        # Keeping the hardship flag but looking at pre-loan metrics
        'hardship_type',
        'hardship_reason',
        'deferral_term',
        'hardship_amount',
        'hardship_start_date',
        'hardship_end_date',
        'payment_plan_start_date',
        'hardship_length',
        'hardship_dpd',
        'hardship_loan_status',
    
        # Trying to extrapolate beyond certain issuing dates of loans
        'issue_d',
        'mths_since_last_major_derog',
        'mths_since_last_record',
        'num_tl_30dpd',
        'num_tl_120dpd_2m',
        
        'debt_settlement_flag',
        'debt_settlement_flag_date',
        'desc',
    
        # This is for investors / loan funding
        'initial_list_status',
        'funded_amnt,
        'funded_amnt_inv',              
        # Probably useful metrics, but all NaN in this dataset              
        'open_acc_6m',                 
        'open_il_12m',
        'open_il_24m',                     
        'open_act_il',   
        'open_rv_12m',
        'open_rv_24m',
                      
        'out_prncp', 
        'out_prncp_inv',
        'policy_code',
        'recoveries',
        # Columns related to debt settlement; we only care about pre loan features              
        'settlement_status',
        'settlement_date',
        'settlement_amount',
        'settlement_percentage',
        'settlement_term',
        'title',
        'total_cu_tl',
                      
        'total_pymnt',
        'total_pymnt_inv',
        'url',
    
        'zip_code'])



SyntaxError: EOL while scanning string literal (<ipython-input-149-a243f9eb9a0d>, line 45)

In [150]:
df.shape

(235631, 145)

In [151]:
df.sample(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
210937,NaN,NaN,14000.0,14000.0,14000.0,36 months,11.99%,464.94,B,B3,"Implementation/Conversion Analyst, Sr",4 years,MORTGAGE,69000.00,Not Verified,Feb-2014,Fully Paid,n,NaN,Borrower added on 02/17/14 > Pay off and IRS...,debt_consolidation,Debt consolidation,720xx,AR,16.64,0.0,Jun-2003,0.0,38.0,NaN,9.0,0.0,20242.0,55.5%,26.0,w,0.0,0.0,16472.710000,16472.71,14000.00,2472.71,0.0,0.00,0.0000,Mar-2016,5314.15,NaN,Mar-2019,0.0,50.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,119672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36500.0,NaN,NaN,NaN,2.0,13297.0,16257.0,27.1,0.0,0.0,127.0,128.0,5.0,5.0,2.0,5.0,50.0,NaN,45.0,3.0,3.0,4.0,5.0,14.0,6.0,6.0,18.0,4.0,9.0,0.0,0.0,0.0,1.0,61.5,40.0,0.0,0.0,164983.0,43138.0,22300.0,36428.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
91983,NaN,NaN,5950.0,5950.0,5950.0,36 months,13.35%,201.49,C,C2,PROPERTY MANAGER,4 years,MORTGAGE,62500.00,Not Verified,Aug-2014,Fully Paid,n,NaN,NaN,home_improvement,Home improvement,800xx,CO,3.55,0.0,Nov-2003,0.0,26.0,NaN,7.0,0.0,1136.0,31.6%,16.0,w,0.0,0.0,7089.770000,7089.77,5950.00,1139.77,0.0,0.00,0.0000,Aug-2016,2455.50,NaN,Jan-2017,0.0,26.0,1.0,Individual,NaN,NaN,NaN,0.0,70.0,128197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3600.0,NaN,NaN,NaN,1.0,18314.0,68.0,88.7,0.0,0.0,129.0,101.0,17.0,17.0,3.0,17.0,26.0,11.0,26.0,1.0,2.0,4.0,2.0,4.0,5.0,4.0,7.0,4.0,7.0,0.0,0.0,0.0,0.0,73.3,100.0,0.0,0.0,137238.0,7399.0,600.0,5805.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
93748,NaN,NaN,12000.0,12000.0,12000.0,36 months,7.69%,374.33,A,A4,eC

In [123]:
#df[df['num_tl_120dpd_2m'] >= 1][['num_tl_30dpd','num_tl_120dpd_2m','num_tl_90g_dpd_24m']]

In [47]:
# df[['int_rate','grade','sub_grade']].sort_values(by = 'int_rate')

In [173]:
# Clean columns
df['term'] = df['term'].str.replace('36 months','short').str.replace('60 months','long')
df['int_rate'] = df['int_rate'].astype(str).str[:-1]